# **1-  算法理解**

借用之前看到的一个例子来阐述EM算法。一个班里有50个男生，50个女生，且男生站左，女生站右。我们假定男生的身高服从正态分布：$N({\mu}_1,\sigma^2_1)$ ，女生的身高则服从另一个正态分布： $N(\mu_2,\sigma^2_2)$ 。这时候我们可以用极大似然法（MLE），分别通过这50个男生和50个女生的样本来估计这两个正态分布的参数。
但现在我们让情况复杂一点，就是这50个男生和50个女生混在一起了。我们拥有100个人的身高数据，却不知道这100个人每一个是男生还是女生。
这时候情况就有点尴尬，因为通常来说，我们只有知道了精确的男女身高的正态分布参数我们才能知道每一个人更有可能是男生还是女生。但从另一方面去考量，我们只有知道了每个人是男生还是女生才能尽可能准确地估计男女各自身高的正态分布的参数。
这个时候有人就想到我们必须从某一点开始，并用迭代的办法去解决这个问题：我们先设定男生身高和女生身高分布的几个参数（初始值），然后根据这些参数去判断每一个样本（人）是男生还是女生，之后根据标注后的样本再反过来重新估计参数。之后再多次重复这个过程，直至稳定。这个算法也就是EM算法。

# **2- 抛硬币问题：**

假设有两枚硬币A、B，以相同的概率随机选择一个硬币，进行如下的抛硬币实验：共做5次实验，每次实验独立的抛十次，结果如图中a所示，例如某次实验产生了H、T、T、T、H、H、T、H、T、H，H代表正面朝上。 
假设试验数据记录员可能是实习生，业务不一定熟悉，造成a和b两种情况 
a表示实习生记录了详细的试验数据，我们可以观测到试验数据中每次选择的是A还是B 
b表示实习生忘了记录每次试验选择的是A还是B，我们无法观测实验数据中选择的硬币是哪个 
问在两种情况下分别如何估计两个硬币正面出现的概率？
如下图： 
![1.png](1.png)

![](2.png)

## **2-1 情况1：已经结果来自A硬币还是B硬币**

对于已知是A硬币还是B硬币抛出的结果的时候，可以直接采用概率的求法来进行求解。

更加严禁地说是根据极大似然估计：
$$
\underset{\theta }{argmax}logP(Y|\theta) = log((\theta_B^5(1-\theta_B)^5) (\theta_A^9(1-\theta_A))(\theta_A^8(1-\theta_A)^2) (\theta_B^4(1-\theta_B)^6) (\theta_A^7(1-\theta_A)^3) ) = log(   (\theta_A^{24}(1-\theta_A)^6) (\theta_B^9(1-\theta_B)^{11})  )
$$

上面这个式子求导之后就能得出$\hat{\theta_A} = \frac{24}{24 + 6} = 0.80$以及$\hat{\theta_B} = \frac{9}{9 + 11} = 0.45$。

## **2-2 情况2：结果来自A或是B硬币未知**

对于含有隐变量的情况，也就是不知道到底是A硬币抛出的结果还是B硬币抛出的结果的时候，就需要采用EM算法进行求解了。

其中的EM算法的第一步就是初始化的过程，然后根据这个参数得出应该产生的结果。

具体过程：

**E步：**
计算在给定的$\hat{\theta_A^{(0)}}$和$\hat{\theta_B^{(0)}}$下，选择的硬币可能是A or B的概率。

例如第一次实验中选择A的概率为
$$
P(z=A|y_1, \theta) = \frac {P(z=A, y_1|\theta)}{P(z=A,y_1|\theta) + P(z=B,y_1|\theta)} = \frac{(0.6)^5*(0.4)^5}{(0.6)^5*(0.4)^5+(0.5)^{10}} = 0.45
$$

则选择B的概率是1-0.45=0.55

那么说明，0.45的概率拿到A，0.55的概率拿到B，之后就在这个条件下做正反面的计算。
即，第一次的试验，0.45的概率是A，0.55的概率是B。那么第一次的试验，A贡献H的次数是0.45*5=2.25H，贡献T的次数是0.45*5=2.25T；

为与原图一致，这里可以小数点后保留一位。

B贡献的H次数是0.55*5=2.75H,贡献的T次数=0.55*5=2.75T;

同理计算第2次，拿A的概率是0.8，拿B的概率是0.2。

A：贡献0.8*9=7.2H;0.8*1=0.8T

B：贡献0.2*9=1.8H;0.2*1=0.2T

逐次计算，得出结果如下图：
![](3.png)


**M步：**

就是根据已知的这些信息对初始化猜测的参数进行修正。
根据上图结果可以看出，A贡献了21.3H,8.6T；
B贡献了17.7H,8.4T；

**那么:**
$\hat{\theta_A^{(1)}}$=21.3/(21.3+8.6)=0.71

$\hat{\theta_B^{(1)}}$=17.7/(17.7+8.4)=0.58

**从Q函数的角度来分析：**

针对Q函数求导，在本题中Q函数形式如下，参数设置参照例1，只是这里的$y_j$代表的是每次正面朝上的个数。

$$
Q(\theta, \theta^i) = \sum_{j=1}^N \sum_{z} P(z|y_j, \theta^i)logP(y_j, z|\theta)=\sum_{j=1}^N [ \mu_jlog(\theta_A^{y_j}(1-\theta_A)^{10-y_j}) + (1-\mu_j)log(\theta_B^{y_j}(1-\theta_B)^{10-y_j})]
$$

从而针对这个式子来对参数求导，例如对$\theta_A$求导:

$$
\frac{\partial Q}{\partial \theta_A} = \mu_1(\frac{y_1}{\theta_A}-\frac{10-y_1}{1-\theta_A}) + \cdot \cdot \cdot  + \mu_5(\frac{y_5}{\theta_A}-\frac{10-y_5}{1-\theta_A}) = \mu_1(\frac{y_1 - 10\theta_A} {\theta_A(1-\theta_A)}) + \cdot \cdot \cdot +\mu_5(\frac{y_5 - 10\theta_A} {\theta_A(1-\theta_A)})  = \frac{\sum_{j=1}^5 \mu_jy_j - \sum_{j=1}^510\mu_j\theta_A} {\theta_A(1-\theta_A)}
$$

求导等于0之后就可得到图中的第一次迭代之后的参数值：

$\hat{\theta_A^{(1)}} = 0.71$

同理对$\theta_B$求导，得出：$\hat{\theta_B^{(1)}} = 0.58$

In [1]:
#-*- coding: utf-8 -*-
import numpy
import scipy
import scipy.stats

#硬币投掷结果
observations = numpy.array([[1,0,0,0,1,1,0,1,0,1],
                        [1,1,1,1,0,1,1,1,1,1],
                        [1,0,1,1,1,1,1,0,1,1],
                        [1,0,1,0,0,0,1,1,0,0],
                        [0,1,1,1,0,1,1,1,0,1]])

def em_single(priors,observations):

    """
    EM算法的单次迭代
    Arguments
    ------------
    priors:[theta_A,theta_B]
    observation:[m X n matrix]

    Returns
    ---------------
    new_priors:[new_theta_A,new_theta_B]
    :param priors:
    :param observations:
    :return:
    """
    counts = {'A': {'H': 0, 'T': 0}, 'B': {'H': 0, 'T': 0}}
    theta_A = priors[0]
    theta_B = priors[1]
    #E step
    for observation in observations:#遍历观测数据
        len_observation = len(observation)#每次的观测数据，10
        num_heads = observation.sum()
        num_tails = len_observation-num_heads
        #二项分布求解公式
        #stats.binom.pmf计算每次观测的概率分布函数。
        #试验结果为1的次数，试验次数，单次出现1的概率。
        #如果第一个参数是一个list，则返回也是一个list
        #例如：
        #stats.binom.pmf(range(6), 5, 1/6.0)
        #运行结果是：
        #array([  4.01877572e-01,   4.01877572e-01,   1.60751029e-01,
        #3.21502058e-02,   3.21502058e-03,   1.28600823e-04])
        #由结果可知：出现0或1次6点的概率为40.2%，而出现3次6点的概率为3.215%。
        contribution_A = scipy.stats.binom.pmf(num_heads,len_observation,theta_A)
        contribution_B = scipy.stats.binom.pmf(num_heads,len_observation,theta_B)

        #将两个概率正规化，得到数据来自硬币A，B的概率：
        weight_A = contribution_A / (contribution_A + contribution_B)
        weight_B = contribution_B / (contribution_A + contribution_B)
        #更新在当前参数下A，B硬币产生的正反面次数
        counts['A']['H'] += weight_A * num_heads
        counts['A']['T'] += weight_A * num_tails
        counts['B']['H'] += weight_B * num_heads
        counts['B']['T'] += weight_B * num_tails

    # M step
    new_theta_A = counts['A']['H'] / (counts['A']['H'] + counts['A']['T'])
    new_theta_B = counts['B']['H'] / (counts['B']['H'] + counts['B']['T'])
    return [new_theta_A,new_theta_B]

def em(observations,prior,tol = 1e-6,iterations=10000):
    """
    EM算法
    ：param observations :观测数据
    ：param prior：模型初值
    ：param tol：迭代结束阈值
    ：param iterations：最大迭代次数
    ：return：局部最优的模型参数
    """
    iteration = 0;
    while iteration < iterations:
        new_prior = em_single(prior,observations)
        delta_change = numpy.abs(prior[0]-new_prior[0])
        if delta_change < tol:
            break
        else:
            prior = new_prior
            iteration +=1
    return [new_prior,iteration]

#假设A硬币向上的概率是0.6，B硬币向上的概率是0.5
rsp = em(observations,[0.6,0.5])#迭代14次就返回了
#[[0.79678875938310978, 0.51958393567528027], 14]，与理论计算结果非常接近，可以再调整误差范围
print(rsp)

[[0.79678875938310978, 0.51958393567528027], 14]


# **3-三硬币模型**

## **3-1 第一种三硬币模型**
假设有三枚硬币A、B、C，每个硬币正面出现的概率是$π、p、q$。进行如下的掷硬币实验：先掷硬币A，正面向上选B，反面选C；然后掷选择的硬币，正面记1，反面记0。独立的进行10次实验，结果如下：1，1，0，1，0，0，1，0，1，1。假设只能观察最终的结果(0 or 1)，而不能观测掷硬币的过程(不知道选的是B or C)，问如何估计三硬币的正面出现的概率？

首先针对某个输出y值，它在参数$\theta (\theta=(\pi, p, q))$下的概率分布为：
$$
P(y|\theta )=\sum_{z}P(y,z|\theta)=\sum_{z}P(z|\theta)P(y|z, \theta) = \pi p^y (1-p)^{1-y} + (1-\pi) q^y (1-q)^{1-y}
$$

从而针对观测数据$Y=(y_1, y_2, \cdot\cdot\cdot, y_n)^T$的似然函数为:
$$
P(Y|\theta ) =\sum_{z}P(Y,z|\theta)=\sum_{z}P(z|\theta)P(Y|z, \theta) = \prod _{j=1} ^{n} [ \pi p^y_j (1-p)^{1-y_j} + (1-\pi) q^y_j (1-q)^{1-y_j}]
$$

因此本题的目标是求解参数$θ$的极大似然估计，即$\hat{\theta} = \underset{\theta }{argmax}logP(Y|\theta)$。
直接对连乘的似然函数求导太复杂，所以一般用极大似然估计都会转化成对数似然函数，连乘转化成了求和，如果这个式子对某个参数(例如$π$)求导，由于这个式子中有“和的对数”，求导非常复杂。因此这个问题需要用EM算法来求解。

**E步:**

根据EM算法，在这一步需要计算的是未观测数据的条件概率分布，也就是每一个$P(z|y_j, \theta)$，$\mu^{i+1}$表示在已知的模型参数$\theta^i$下观测数据$y_j$来自掷硬币B的概率，相应的来自掷C的概率就是$1-\mu^{i+1}$。
$$
\mu ^{i+1} = \frac {\pi^i ({p^i})^{y_j}(1-p^i)^{1-y_j}} {\pi^i ({p^i})^{y_j}(1-p^i)^{1-y_j} + (1-\pi^i) ({q^i})^{y_j} (1-q^i)^{1-y_j}}
$$

这里的分子就是z取掷硬币B和y的联合概率分布，需要注意的是，这里的$1-\mu^{i+1}$通过E步的计算就已经是一个常数了，后面的求导不需要把这个式子代入。

**M步:**

针对Q函数求导，Q函数的表达式是:
$$
Q(\theta, \theta^i) = \sum_{j=1}^N \sum_{z} P(z|y_j, \theta^i)logP(y_j, z|\theta)=\sum_{j=1}^N \mu_jlog(\pi p^{y_j}(1-p)^{1-y_j}) + (1-\mu_j)log((1-\pi) q^{y_j} (1-q)^{1-y_j})]
$$
对$\pi$求导：

$$
\frac{\partial Q}{\partial \pi} = (\frac{\mu_1}{\pi} - \frac{1-\mu_1}{1-\pi})+\cdot \cdot \cdot + (\frac{\mu_N}{\pi} - \frac{1-\mu_N}{1-\pi}) = \frac{\mu_1-\pi}{\pi(1-\pi)} + \cdot \cdot \cdot + \frac{\mu_N-\pi}{\pi(1-\pi)} = \frac{\sum _{j=1} ^N\mu_j-N\pi}{\pi(1-\pi)}
$$

再令这个结果等于0，即获得$\pi^{i+1} = \frac{1}{N}\sum_{j=1}^{N}\mu_j^{i+1}$，其他两个也同理。

In [2]:
# -*- coding:utf-8 -*-
'''
三模型问题
'''
import math

[pi,p,q]=[0.4,0.6,0.7]

y=[1,1,0,1,0,0,1,0,1,1]

#书中公式
def cal_u(pi1,p1,q1,yi):
    return pi1*math.pow(p1,yi)*math.pow(1-p1,1-yi)/float(pi1*math.pow(p1,yi)*math.pow(1-p1,1-yi)+(1-pi1)*math.pow(q1,yi)*math.pow(1-q1,1-yi))

def e_step(pi1,p1,q1,y):
    '''
    E步就是计算u
    :param pi1:
    :param p1:
    :param q1:
    :param y:
    :return:
    '''
    return [cal_u(pi1,p1,q1,yi) for yi in y]

def m_step(u,y):
    pi1=sum(u)/len(u)
    p1=sum([u[i]*y[i] for i in range(len(u))]) / sum(u)
    q1=sum([(1-u[i])*y[i] for i in range(len(u))]) / sum([1-u[i] for i in range(len(u))])
    return [pi1,p1,q1]

def run(start_y,start_pi,start_p,start_q,iter_num):
    for i in range(iter_num):
        u = e_step(start_pi,start_p,start_q,y)
        #print("len(u)=",len(u))
        print(i,[start_pi,start_p,start_q])
        if [start_pi,start_p,start_q]==m_step(u,y):
            break
        else:
            [start_pi,start_p,start_q]=m_step(u,y)

run(y,pi,p,q,100)
#输出结果
#[0.40641711229946537, 0.536842105263158, 0.6432432432432431]

0 [0.4, 0.6, 0.7]
1 [0.40641711229946526, 0.5368421052631579, 0.6432432432432431]
2 [0.40641711229946537, 0.5368421052631579, 0.6432432432432431]
3 [0.40641711229946537, 0.536842105263158, 0.6432432432432431]


## **3-2 第二种三硬币模型**

假设有三枚硬币A、B、C，每个硬币正面出现的概率是$π、p、q$。进行如下的掷硬币实验：先掷硬币A，正面向上选B，反面选C；然后对选出的硬币进行一组实验，独立的抛十次。共做5次实验，每次实验独立的抛十次，结果如下图所示:
![](4.png)

例如某次实验产生了H、T、T、T、H、H、T、H、T、H。H代表正面朝上，1，T为反面，0。假设只能观察最终的结果(0 or 1)，而不能观测掷硬币的过程(不知道选的是B or C)，问如何估计三硬币的正面出现的概率？

这个问题是3-1和2-2问题的结合。

In [3]:
# -*- coding:utf-8 -*-
'''
结合第1种情况和第2种情况。
'''
import numpy as np
from scipy import stats

def em_single(priors, observations):
    """
    EM算法单次迭代
    Arguments
    ---------
    priors : [theta_A, theta_B，theta_C]
    observations : [m X n matrix]

    Returns
    --------
    new_priors: [new_theta_A, new_theta_B,new_theta_C]
    :param priors:
    :param observations:
    :return:
    """
    counts = {'A': {'H': 0, 'T': 0}, 'B': {'H': 0, 'T': 0}}
    theta_A = priors[0]
    theta_B = priors[1]
    theta_c=priors[2]

    # E step
    weight_As=[]
    for observation in observations:
        len_observation = len(observation)
        num_heads = observation.sum()
        num_tails = len_observation - num_heads
        contribution_A = theta_c*stats.binom.pmf(num_heads, len_observation, theta_A)
        contribution_B = (1-theta_c)*stats.binom.pmf(num_heads, len_observation, theta_B)  # 两个二项分布
        weight_A = contribution_A / (contribution_A + contribution_B)
        weight_B = contribution_B / (contribution_A + contribution_B)
        # 更新在当前参数下A、B硬币产生的正反面次数
        weight_As.append(weight_A)
        counts['A']['H'] += weight_A * num_heads
        counts['A']['T'] += weight_A * num_tails
        counts['B']['H'] += weight_B * num_heads
        counts['B']['T'] += weight_B * num_tails

    # M step
    new_theta_c = 1.0*sum(weight_As)/len(weight_As)
    new_theta_A = counts['A']['H'] / (counts['A']['H'] + counts['A']['T'])
    new_theta_B = counts['B']['H'] / (counts['B']['H'] + counts['B']['T'])
    return [new_theta_A, new_theta_B,new_theta_c]

def em(observations, prior, tol=1e-6, iterations=10000):
    """
    EM算法
    :param observations: 观测数据
    :param prior: 模型初值
    :param tol: 迭代结束阈值
    :param iterations: 最大迭代次数
    :return: 局部最优的模型参数
    """
    import math
    iteration = 0
    while iteration < iterations:
        new_prior = em_single(prior, observations)
        delta_change = np.abs(prior[0] - new_prior[0])
        if delta_change < tol:
            break
        else:
            prior = new_prior
            iteration += 1
    return [new_prior, iteration]

# 硬币投掷结果观测序列：1表示正面，0表示反面。
observations = np.array([[1, 0, 0, 0, 1, 1, 0, 1, 0, 1],
                         [1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
                         [1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
                         [1, 0, 1, 0, 0, 0, 1, 1, 0, 0],
                         [0, 1, 1, 1, 0, 1, 1, 1, 0, 1]])

print(em(observations, [0.5, 0.8, 0.6]))
#运行结果：
#[[0.51392121603987106, 0.79337052912023864, 0.47726196801164544], 42]

[[0.51392121603987106, 0.79337052912023864, 0.47726196801164544], 42]
